- Keys in file: ['Day_1', 'Meta']
- Keys inside Meta: ['Age', 'Code', 'Height', 'No. Days', 'Sex', 'Weight']
- Keys inside Day_1: ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28', 'Trial_29', 'Trial_30', 'Trial_31', 'Trial_32', 'Trial_33', 'Trial_34', 'Trial_35', 'Trial_36']
- Keys inside Trial_01 : ['Acc_Left_Foot', 'Acc_Left_Shank', 'Acc_Left_Thigh', 'Acc_Pelvis', 'Acc_Right_Foot', 'Acc_Right_Shank', 'Acc_Right_Thigh', 'Acc_T8', 'Ang_L5S1', 'Ang_Left_Ankle', 'Ang_Left_Hip', 'Ang_Left_Knee', 'Ang_Right_Ankle', 'Ang_Right_Hip', 'Ang_Right_Knee', 'EMG_Left_BF', 'EMG_Left_GM', 'EMG_Left_Gmax', 'EMG_Left_Gmed', 'EMG_Left_RF', 'EMG_Left_ST', 'EMG_Left_TA', 'EMG_Left_VL', 'EMG_Right_BF', 'EMG_Right_GM', 'EMG_Right_Gmax', 'EMG_Right_Gmed', 'EMG_Right_RF', 'EMG_Right_ST', 'EMG_Right_TA', 'EMG_Right_VL', 'Gyr_Left_Foot', 'Gyr_Left_Shank', 'Gyr_Left_Thigh', 'Gyr_Pelvis', 'Gyr_Right_Foot', 'Gyr_Right_Shank', 'Gyr_Right_Thigh', 'Gyr_T8', 'Label', 'Markers', 'Time']

=> here, we extract only the EMG contents: ones that start with "EMG_ ..."


In [3]:
import h5py
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../../Shared')
import processing, Model, Visualization
from sklearn.model_selection import train_test_split
import modules
import random


def Run_feature_wise(F_name):
    base_path = 'D:/Data/Gait-EMG/Mypredict/'
    f = h5py.File(base_path+f"{F_name}", "r")

    key_lst = list(f['Day_1'].keys())
    print("Trials: ", key_lst)

    Features = ['mav', 'var', 'zc', 'iemg', 'wl', 'wamp', 'mavs', 'rms', 'ssc', 'msq', 'v3', 'ld', 'dabs', 'mfl', 'mpr', 'mnf', 'psr', 'arc1', 'arc2', 'arc3', 'arc4', 'cc1', 'cc2', 'cc3', 'cca', 'dwtc1', 'dwtc2', 'dwtpc1', 'dwtpc2', 'dwtpc3']

    Acc_all = []
    ten_files = random.choices(key_lst, k=5)

    for trials in ten_files:
        print("\n\n===================================", trials, "===================================\n")
        trial = f["Day_1"][trials]

        label, valid_idx = modules.label_extraction(trial)
        emg_keys = [k for k in trial.keys() if k.startswith("EMG_")]
        X_emg = modules.stack_signals(trial, emg_keys, valid_idx)

        balanced_idx = modules.Downsample_to_balance_class(label)
        X_emg_bal, y_bal = X_emg[balanced_idx], label[balanced_idx]
        print("EMG:", X_emg_bal.shape, "y:", y_bal.shape)
        #X_emg_bal, y_bal = random_downsample(X_emg_bal, y_bal, fraction=0.5)
        #print("After downsampling - EMG:", X_emg_bal.shape, "y:", y_bal.shape)

        window_size, step_size = 200, 10
        num_channels, num_features = X_emg.shape[1], 30
        X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal,  window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

        all_X, all_y = modules.get_X_y(X_tmp, y_tmp)
        all_y = modules.y_change_to_int(all_y)
        all_X = all_X.reshape(-1, num_channels, num_features, 1)

        Sub_acc = []
        for feature_idx in range(num_features):
            X_feature = all_X[:, :, feature_idx]   # shape: (samples, num_channels, 1, 1)
            X_feature = X_feature.reshape(-1, num_channels, 1)
            X_train, X_test, y_train, y_test = train_test_split(X_feature, all_y, test_size=0.2, random_state=42)

            model = Model.build_model_1D(input_shape=X_feature.shape[1:], num_classes=len(np.unique(all_y)))
            history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=512, verbose=0)

            print(f"\t {trials} - Feature {feature_idx} ({Features[feature_idx]}): "
                f"val acc => {np.max(history.history['val_accuracy'])*100:.2f}%")
            Sub_acc.append(np.max(history.history['val_accuracy'])*100)
        Acc_all.append(Sub_acc)


    pd.DataFrame(Acc_all, columns=Features).to_csv(f"./feature_wise_{F_name}.csv", index=False)

In [2]:
Run_feature_wise(F_name="MP101.hdf5")



=================================== Trial_01 ===================================

EMG: (40500, 16) y: (40500,)
Unique labels: [0.  1.  2.  3.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
	 Trial_01 - Feature 0 (mav): val acc => 36.31%
	 Trial_01 - Feature 1 (var): val acc => 22.43%
	 Trial_01 - Feature 2 (zc): val acc => 90.83%
	 Trial_01 - Feature 3 (iemg): val acc => 95.66%
	 Trial_01 - Feature 4 (wl): val acc => 98.14%
	 Trial_01 - Feature 5 (wamp): val acc => 55.27%
	 Trial_01 - Feature 6 (mavs): val acc => 6.32%
	 Trial_01 - Feature 7 (rms): val acc => 43.12%
	 Trial_01 - Feature 8 (ssc): val acc => 9.42%
	 Trial_01 - Feature 9 (msq): val acc => 27.14%
	 Trial_01 - Feature 10 (v3): val acc => 9.54%
	 Trial_01 - Feature 11 (ld): val acc => 20.07%
	 Trial_01 - Feature 12 (dabs): val acc => 36.43%
	 Trial_01 - Feature 13 (mfl): val acc => 11.15%
	 Trial_01 - Feature 14 (mpr): val acc => 29.86%
	 Trial_01 - Feature 15 (mnf)


KeyboardInterrupt



In [4]:
Run_feature_wise(F_name="MP102.hdf5")

Trials:  ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28', 'Trial_29', 'Trial_30', 'Trial_31', 'Trial_32']


=================================== Trial_15 ===================================

EMG: (25500, 16) y: (25500,)
	 Trial_15 - Feature 0 (mav): val acc => 9.66%
	 Trial_15 - Feature 1 (var): val acc => 5.72%
	 Trial_15 - Feature 2 (zc): val acc => 30.57%
	 Trial_15 - Feature 3 (iemg): val acc => 18.93%
	 Trial_15 - Feature 4 (wl): val acc => 22.49%
	 Trial_15 - Feature 5 (wamp): val acc => 11.05%
	 Trial_15 - Feature 6 (mavs): val acc => 6.71%
	 Trial_15 - Feature 7 (rms): val acc => 6.71%
	 Trial_15 - Feature 8 (ssc): val acc => 8.88%
	 Trial_15 - Feature 9 (msq): val acc => 5.33%
	 Trial_15 

In [5]:
Run_feature_wise(F_name="MP103.hdf5")

Trials:  ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28', 'Trial_29', 'Trial_30', 'Trial_31', 'Trial_32', 'Trial_33', 'Trial_34', 'Trial_35', 'Trial_36', 'Trial_37', 'Trial_38', 'Trial_39', 'Trial_40', 'Trial_41', 'Trial_42', 'Trial_43', 'Trial_44', 'Trial_45']


=================================== Trial_33 ===================================

EMG: (27200, 16) y: (27200,)
	 Trial_33 - Feature 0 (mav): val acc => 10.91%
	 Trial_33 - Feature 1 (var): val acc => 6.28%
	 Trial_33 - Feature 2 (zc): val acc => 19.22%
	 Trial_33 - Feature 3 (iemg): val acc => 23.48%
	 Trial_33 - Feature 4 (wl): val acc => 45.29%
	 Trial_33 - Feature 5 (wamp): val acc => 15.90%
	 Trial_33 - Feature 6 (mavs): val acc => 5

In [6]:
Run_feature_wise(F_name="MP104.hdf5")

Trials:  ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27']


=================================== Trial_04 ===================================

EMG: (35700, 16) y: (35700,)
	 Trial_04 - Feature 0 (mav): val acc => 6.75%
	 Trial_04 - Feature 1 (var): val acc => 8.72%
	 Trial_04 - Feature 2 (zc): val acc => 27.57%
	 Trial_04 - Feature 3 (iemg): val acc => 24.75%
	 Trial_04 - Feature 4 (wl): val acc => 25.04%
	 Trial_04 - Feature 5 (wamp): val acc => 23.91%
	 Trial_04 - Feature 6 (mavs): val acc => 6.47%
	 Trial_04 - Feature 7 (rms): val acc => 9.99%
	 Trial_04 - Feature 8 (ssc): val acc => 13.64%
	 Trial_04 - Feature 9 (msq): val acc => 14.21%
	 Trial_04 - Feature 10 (v3): val acc => 11.95%
	 Trial_04 - Feature 

In [7]:
Run_feature_wise(F_name="MP105.hdf5")

Trials:  ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28', 'Trial_29', 'Trial_30', 'Trial_31', 'Trial_32', 'Trial_33', 'Trial_34', 'Trial_35', 'Trial_36', 'Trial_37']


=================================== Trial_30 ===================================

EMG: (32300, 16) y: (32300,)
	 Trial_30 - Feature 0 (mav): val acc => 7.47%
	 Trial_30 - Feature 1 (var): val acc => 7.47%
	 Trial_30 - Feature 2 (zc): val acc => 27.84%
	 Trial_30 - Feature 3 (iemg): val acc => 41.99%
	 Trial_30 - Feature 4 (wl): val acc => 27.99%
	 Trial_30 - Feature 5 (wamp): val acc => 55.05%
	 Trial_30 - Feature 6 (mavs): val acc => 6.07%
	 Trial_30 - Feature 7 (rms): val acc => 14.15%
	 Trial_30 - Feature 8 (ssc): val acc => 9.1

KeyError: "Unable to synchronously open object (object 'Label' doesn't exist)"

In [8]:
Run_feature_wise(F_name="MP106.hdf5")

Trials:  ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28']


=================================== Trial_08 ===================================

EMG: (30600, 16) y: (30600,)
	 Trial_08 - Feature 0 (mav): val acc => 11.99%
	 Trial_08 - Feature 1 (var): val acc => 6.57%
	 Trial_08 - Feature 2 (zc): val acc => 28.24%
	 Trial_08 - Feature 3 (iemg): val acc => 13.96%
	 Trial_08 - Feature 4 (wl): val acc => 34.32%
	 Trial_08 - Feature 5 (wamp): val acc => 40.56%
	 Trial_08 - Feature 6 (mavs): val acc => 7.55%
	 Trial_08 - Feature 7 (rms): val acc => 8.21%
	 Trial_08 - Feature 8 (ssc): val acc => 14.29%
	 Trial_08 - Feature 9 (msq): val acc => 7.22%
	 Trial_08 - Feature 10 (v3): val acc => 7.22%
	 Trial_08

In [9]:
Run_feature_wise(F_name="MP107.hdf5")

Trials:  ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28', 'Trial_29', 'Trial_30', 'Trial_31', 'Trial_32', 'Trial_33', 'Trial_34', 'Trial_35', 'Trial_36', 'Trial_37']


=================================== Trial_23 ===================================

EMG: (30600, 16) y: (30600,)
	 Trial_23 - Feature 0 (mav): val acc => 5.58%
	 Trial_23 - Feature 1 (var): val acc => 6.73%
	 Trial_23 - Feature 2 (zc): val acc => 34.15%
	 Trial_23 - Feature 3 (iemg): val acc => 20.20%
	 Trial_23 - Feature 4 (wl): val acc => 46.47%
	 Trial_23 - Feature 5 (wamp): val acc => 49.59%
	 Trial_23 - Feature 6 (mavs): val acc => 6.08%
	 Trial_23 - Feature 7 (rms): val acc => 12.81%
	 Trial_23 - Feature 8 (ssc): val acc => 9.3

In [10]:
Run_feature_wise(F_name="MP108.hdf5")

Trials:  ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28', 'Trial_29', 'Trial_30', 'Trial_31', 'Trial_32', 'Trial_33', 'Trial_34', 'Trial_35', 'Trial_36']


=================================== Trial_26 ===================================

EMG: (34000, 16) y: (34000,)
	 Trial_26 - Feature 0 (mav): val acc => 5.76%
	 Trial_26 - Feature 1 (var): val acc => 10.04%
	 Trial_26 - Feature 2 (zc): val acc => 36.63%
	 Trial_26 - Feature 3 (iemg): val acc => 35.16%
	 Trial_26 - Feature 4 (wl): val acc => 34.27%
	 Trial_26 - Feature 5 (wamp): val acc => 50.37%
	 Trial_26 - Feature 6 (mavs): val acc => 5.76%
	 Trial_26 - Feature 7 (rms): val acc => 7.53%
	 Trial_26 - Feature 8 (ssc): val acc => 14.48%
	 Trial_

In [11]:
Run_feature_wise(F_name="MP109.hdf5")

Trials:  ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28', 'Trial_29', 'Trial_30', 'Trial_31', 'Trial_32', 'Trial_33', 'Trial_34', 'Trial_35', 'Trial_36']


=================================== Trial_01 ===================================

EMG: (25500, 16) y: (25500,)
	 Trial_01 - Feature 0 (mav): val acc => 7.69%
	 Trial_01 - Feature 1 (var): val acc => 6.51%
	 Trial_01 - Feature 2 (zc): val acc => 19.72%
	 Trial_01 - Feature 3 (iemg): val acc => 9.66%
	 Trial_01 - Feature 4 (wl): val acc => 11.24%
	 Trial_01 - Feature 5 (wamp): val acc => 15.38%
	 Trial_01 - Feature 6 (mavs): val acc => 6.51%
	 Trial_01 - Feature 7 (rms): val acc => 8.09%
	 Trial_01 - Feature 8 (ssc): val acc => 8.68%
	 Trial_01 

In [12]:
Run_feature_wise(F_name="MP110.hdf5")

Trials:  ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28', 'Trial_29', 'Trial_30', 'Trial_31', 'Trial_32', 'Trial_33', 'Trial_34', 'Trial_35', 'Trial_36']


=================================== Trial_25 ===================================

EMG: (40800, 16) y: (40800,)
	 Trial_25 - Feature 0 (mav): val acc => 11.44%
	 Trial_25 - Feature 1 (var): val acc => 6.15%
	 Trial_25 - Feature 2 (zc): val acc => 29.03%
	 Trial_25 - Feature 3 (iemg): val acc => 43.17%
	 Trial_25 - Feature 4 (wl): val acc => 47.85%
	 Trial_25 - Feature 5 (wamp): val acc => 53.26%
	 Trial_25 - Feature 6 (mavs): val acc => 6.15%
	 Trial_25 - Feature 7 (rms): val acc => 11.93%
	 Trial_25 - Feature 8 (ssc): val acc => 16.85%
	 Trial

In [ ]:
for file in []
pd.read_csv(f'./feature_wise_{file}.csv')